Steps to setup a Project

1. Create Virtual Environment and swith to virtual environment
2. pip install uv
3. uv init
4. uv add ipykernel, langchain, langchain-community, pandas, openpyxl

In [2]:
# Create a Sample data in CSV File
import pandas as pd

data = {
    'Product': ['Laptop', 'Mouse', 'Keyboard', 'Monitor', 'Webcam'],
    'Category': ['Electronics', 'Accessories', 'Accessories', 'Electronics', 'Electronics'],
    'Price': [999.99, 29.99, 79.99, 299.99, 89.99],
    'Stock': [50, 200, 150, 75, 100],
    'Description': [
        'High-performance laptop with 16GB RAM and 512GB SSD',
        'Wireless optical mouse with ergonomic design',
        'Mechanical keyboard with RGB backlighting',
        '27-inch 4K monitor with HDR support',
        '1080p webcam with noise cancellation'
    ]
}

# Save as CSV
df = pd.DataFrame(data)
df.to_csv('products.csv', index=False)

In [5]:
# Create Sample data in Excel File
with pd.ExcelWriter('inventory.xlsx') as writer:
    df.to_excel(writer, sheet_name='Products', index=False)
    
    # Add another sheet
    summary_data = {
        'Category': ['Electronics', 'Accessories'],
        'Total_Items': [3, 2],
        'Total_Value': [1389.97, 109.98]
    }
    pd.DataFrame(summary_data).to_excel(writer, sheet_name='Summary', index=False)

CSV Processing

In [ ]:
from langchain_community.document_loaders import CSVLoader
from langchain_community.document_loaders import UnstructuredCSVLoader

# Method 1: CSVLoader - Each row becomes a document
print("1️⃣ CSVLoader - Row-based Documents")
csv_loader = CSVLoader(
    file_path='products.csv',
    encoding='utf-8',
    csv_args={
        'delimiter': ',',
        'quotechar': '"',
    }
)

csv_docs = csv_loader.load()
print(csv_docs)
print(f"Loaded {len(csv_docs)} documents (one per row)")
print("\nFirst document:")
print(f"Content: {csv_docs[0].page_content}")
print(f"Metadata: {csv_docs[0].metadata}")

In [ ]:
from typing import List
from langchain_core.documents import Document

# Method 2: Custom CSV processing for better control
print("\n2️⃣ Custom CSV Processing")
def process_csv_intelligently(filepath: str) -> List[Document]:
    """Process CSV with intelligent document creation"""
    df = pd.read_csv(filepath)
    documents = []
    
    # Strategy 1: One document per row with structured content
    for idx, row in df.iterrows():
        # Create structured content
        content = f"""Product Information:
        Name: {row['Product']}
        Category: {row['Category']}
        Price: ${row['Price']}
        Stock: {row['Stock']} units
        Description: {row['Description']}"""
        
        # Create document with rich metadata
        doc = Document(
            page_content=content,
            metadata={
                'source': filepath,
                'row_index': idx,
                'product_name': row['Product'],
                'category': row['Category'],
                'price': row['Price'],
                'data_type': 'product_info'
            }
        )
        documents.append(doc)
    return documents

In [ ]:
process_csv_intelligently('products.csv')

Excel Processing

In [ ]:
# Method 1: Using pandas for full control
print("1️⃣ Pandas-based Excel Processing")
def process_excel_with_pandas(filepath: str) -> List[Document]:
    """Process Excel with sheet awareness"""
    documents = []
    
    # Read all sheets
    excel_file = pd.ExcelFile(filepath)
    
    for sheet_name in excel_file.sheet_names:
        df = pd.read_excel(filepath, sheet_name=sheet_name)
        
        # Create document for each sheet
        sheet_content = f"Sheet: {sheet_name}\n"
        sheet_content += f"Columns: {', '.join(df.columns)}\n"
        sheet_content += f"Rows: {len(df)}\n\n"
        sheet_content += df.to_string(index=False)
        
        doc = Document(
            page_content=sheet_content,
            metadata={
                'source': filepath,
                'sheet_name': sheet_name,
                'num_rows': len(df),
                'num_columns': len(df.columns),
                'data_type': 'excel_sheet'
            }
        )
        documents.append(doc)
    
    return documents

In [ ]:
excel_docs = process_excel_with_pandas('inventory.xlsx')
print(f"Processed {len(excel_docs)} sheets")
excel_docs

In [13]:
from langchain_community.document_loaders import UnstructuredExcelLoader
# Method 2: UnstructuredExcelLoader
print("\n2️⃣ UnstructuredExcelLoader")
unstructured_docs = None 
try:
    excel_loader = UnstructuredExcelLoader(
        'inventory.xlsx',
        mode="elements"
    )
    unstructured_docs = excel_loader.load()
    print("  ✅ Handles complex Excel features")
    print("  ✅ Preserves formatting info")
    print("  ❌ Requires unstructured library")
except Exception as e:
    print("  ℹ️ Requires unstructured library with Excel support")


2️⃣ UnstructuredExcelLoader
  ℹ️ Requires unstructured library with Excel support


In [15]:
unstructured_docs